In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.models import Sequential
from keras.layers.core import Dense, Dropout

import sys
sys.path.insert(0, '/home/lcastellazzi/MDM32/src/utils')
from preprocessing import TraceHandler
from nicv import nicv
import constants

In [2]:
paths = {'train': '/prj/side_channel/PinataTraces/CURR/D1-K1_50k_500MHz + Resampled at 168MHz.trs',
         'test': '/prj/side_channel/PinataTraces/CURR/D1-K1_50k_500MHz (NICV) + Resampled at 168MHz.trs'}

trace_handlers = {key: TraceHandler(path) for key, path in paths.items()}

100%|██████████| 50000/50000 [00:21<00:00, 2312.60it/s]


In [3]:
BYTE_IDX = 0
VAL_PERC = 0.1
N_CLASSES = 256

x_train, x_val, y_train, y_val = trace_handlers['train'].generate_train_val(BYTE_IDX, val_perc=VAL_PERC) 
x_test, y_test = trace_handlers['test'].generate_test(BYTE_IDX) 

y_train_cat = to_categorical(y_train, N_CLASSES)
y_val_cat = to_categorical(y_val, N_CLASSES)
y_test_cat = to_categorical(y_test, N_CLASSES)

In [14]:
# MLP from Mind the Portability (code from authors - mlp_no_portability.py)

LAYER_LIST = [5] # List for grid-search
NEURON_LIST = [200] # List for grid-search
NUM_FEATURES = len(x_train[0]) # 1237
BATCH_SIZE = 256
NUM_EPOCHS = 50
LOSS_FUNCTION = 'categorical_crossentropy'
OPTIMIZER = RMSprop(lr=0.001)

validation_scores = np.zeros((len(LAYER_LIST), len(NEURON_LIST)))
all_test_predictions = np.zeros((len(LAYER_LIST),
                                 len(NEURON_LIST), 
                                 y_test_cat.shape[0], 
                                 N_CLASSES))

print(f'Epochs: {NUM_EPOCHS}, batch size: {BATCH_SIZE}, loss: {LOSS_FUNCTION}, optimizer: {OPTIMIZER}')
print('Starting grid search.')

for layer_idx, layer_num in enumerate(LAYER_LIST):
    for neurons_idx, neurons_num in enumerate(NEURON_LIST):
        
        model = Sequential()
        
        # First hidden layer, with explicit input dimensions, ReLu
        model.add(Dense(neurons_num, 
                        activation='relu',
                        input_shape=(NUM_FEATURES, )))
        
        # Add more hidden layers, ReLu
        for _ in range(layer_num - 1):
            model.add(Dense(neurons_num, 
                            activation='relu'))
        
        # Output layer: softmax
        model.add(Dense(N_CLASSES, 
                        activation='softmax'))
            
        # Compile, print and train model
        model.compile(optimizer=OPTIMIZER, 
                      loss=LOSS_FUNCTION,
                      metrics=['accuracy'])
        model.summary()
        print()
        print('TRAIN')
        model.fit(x_train, 
                  y_train_cat,
                  validation_data=(x_val, y_val_cat),
                  epochs=NUM_EPOCHS,
                  verbose=1)
        
        # Add accuracy score to table
        print()
        print('VAL')
        validation_scores[layer_idx, neurons_idx] = model.evaluate(x_val, 
                                                                   y_val_cat, 
                                                                   batch_size=BATCH_SIZE)[1]
        
        all_test_predictions[layer_idx, neurons_idx, :, :] = model.predict(x_test)
        
print()
print('Accuracy scores for validation set:')
print(validation_scores)

best_layer_idx, best_neurons_idx = np.unravel_index(np.argmax(validation_scores), 
                                                    validation_scores.shape)

print()
print(f'Best model has {validation_scores[best_layer_idx, best_neurons_idx]} accuracy on the validation set, using {LAYER_LIST[best_layer_idx]} layers of {NEURON_LIST[best_neurons_idx]} neurons')

predictions = all_test_predictions[best_layer_idx, best_neurons_idx]

Epochs: 50, batch size: 256, loss: categorical_crossentropy, optimizer: <keras.optimizer_v2.rmsprop.RMSprop object at 0x7f0182779f70>
Starting grid search.
Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_89 (Dense)            (None, 200)               247600    
                                                                 
 dense_90 (Dense)            (None, 200)               40200     
                                                                 
 dense_91 (Dense)            (None, 200)               40200     
                                                                 
 dense_92 (Dense)            (None, 200)               40200     
                                                                 
 dense_93 (Dense)            (None, 200)               40200     
                                                                 
 dense_94 (Dense)            

In [16]:
all_test_predictions.shape

(1, 1, 50000, 256)

In [25]:
pred = all_test_predictions[0][0]
res = [np.argmax(pred[i]) for i in range(pred.shape[0])]

In [28]:
classes = range(256)

y_test_res = [classes[single_pred] for single_pred in res]

diff = y_test - y_test_res
diff

array([ -11, -209,  -59, ..., -107,  -46,  -41])